In [4]:
# hacked together approach to access my common functions since i'm short on time.
cd ..

/Users/gagesonntag/hotel-case/hotel-case


In [7]:
import pandas as pd
from scipy.stats import ttest_ind
from scipy.stats import fisher_exact
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\scripts")

from scripts.common_functions import test_financial_metrics

In [9]:
# downloaded Google Sheet provided by SC and saved locally
path = 'data/Thank You Call Data - Sheet1.tsv'
raw_data = pd.read_csv(path, sep = '\t')

In [10]:
# method chain and test this post EDA

filter_criteria = (raw_data.BOOKINGS < 12)& (raw_data.GMV < 8000) & (raw_data.BOOKINGS.notnull())

dataset = (raw_data[filter_criteria]
           .assign(MARGIN = lambda x: x.LTR / x.GMV)
           .assign(CUSTOMER_RETURNED = lambda x: x.BOOKINGS > 1)
           .assign(GMV_PER_BOOKING = lambda x: x.GMV / x.BOOKINGS)
          )

# GMV

Hypothesis is that the thank you call leads to higher lifetimes GMV

In [13]:
_, _, _ = test_financial_metrics(dataset,'GMV')

Campaign lift of $56.92372184541762 (6.0%) vs $944.9203556818181 baseline with p 8.105493833861987e-07


# LTR

In [ ]:
_, _, _ = test_financial_metrics(dataset,'LTR')

# BOOKINGS & GMV PER BOOKING

## Don't use p value results, not relevant for rate data, but leverage lift results for deck


In [14]:
_, _, _ = test_financial_metrics(dataset,'BOOKINGS')

Campaign lift of $0.07476097056046083 (4.9%) vs $1.5334224598930482 baseline with p 3.554089301881193e-05


In [17]:
_, _, _ = test_financial_metrics(dataset,'GMV_PER_BOOKING')

Campaign lift of $8.902639696029155 (1.4%) vs $636.7880225521448 baseline with p 0.0001613761610843776


# Return Rate

In [148]:
return_rate_dataset = (dataset
.groupby(['THANK_YOU_STATUS','CUSTOMER_RETURNED'])
.USER_ID
.count()
.reset_index()
.pivot(index = 'THANK_YOU_STATUS',
 columns = 'CUSTOMER_RETURNED',
 values = 'USER_ID')
)

return_rate_dataset

CUSTOMER_RETURNED,False,True
THANK_YOU_STATUS,,
no_call,11077,3883
thank_you_call,5716,2178


In [149]:
return_rate_results = fisher_exact(return_rate_dataset)

summary_return_rate = (return_rate_dataset
                       .reset_index()
                       .assign(RETURN_RATE = lambda x: x[True] / (x[False] + x[True]))
                      )

control_group_rate = summary_return_rate[summary_return_rate.THANK_YOU_STATUS == 'no_call'][['RETURN_RATE']].sum()[0]
test_group_rate = summary_return_rate[summary_return_rate.THANK_YOU_STATUS == 'thank_you_call'][['RETURN_RATE']].sum()[0]
delta_return_rate = test_group_rate - control_group_rate

print(f'Test saw {100*delta_return_rate:.2}% change in customer return rate from baseline of {100*control_group_rate:.2f}% with p value {return_rate_results[1]}')

Test saw 1.6% change in customer return rate from baseline of 25.96% with p value 0.008115007865450358
